In [1]:
%pip install torch pandas torchvision scikit-learn tqdm kaggle timm -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
# upload kaggle.json first.
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# !apt update -qq
# !apt install -qq unzip
!kaggle datasets download nirmalsankalana/sugarcane-leaf-disease-dataset
!unzip -q sugarcane-leaf-disease-dataset.zip -d data

E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
Dataset URL: https://www.kaggle.com/datasets/nirmalsankalana/sugarcane-leaf-disease-dataset
License(s): CC0-1.0


In [4]:
import os
import shutil

import pandas as pd

# Define paths
data_root = "data"
images_dir = os.path.join(data_root, "images")

# Create images directory if it doesn't exist
os.makedirs(images_dir, exist_ok=True)

# List to store image paths and labels
dataset = []

# Loop through each subfolder
for subfolder in os.listdir(data_root):
    subfolder_path = os.path.join(data_root, subfolder)

    # Ensure it's a directory
    if os.path.isdir(subfolder_path) and subfolder != "images":
        # Loop through images inside the subfolder
        for image in os.listdir(subfolder_path):
            old_image_path = os.path.join(subfolder_path, image)

            # Ensure it's a file (image)
            if os.path.isfile(old_image_path):
                # Define new image path in "data/images" directory
                new_image_path = os.path.join(images_dir, image)

                # If filename already exists, rename it to avoid conflicts
                if os.path.exists(new_image_path):
                    base, ext = os.path.splitext(image)
                    counter = 1
                    while os.path.exists(new_image_path):
                        new_image_path = os.path.join(images_dir, f"{base}_{counter}{ext}")
                        counter += 1

                # Move image
                shutil.move(old_image_path, new_image_path)

                # Append to dataset with updated path and original label
                dataset.append({"image_path": new_image_path, "label": subfolder})

        # Optionally remove empty subfolder after moving images
        os.rmdir(subfolder_path)

df = pd.DataFrame(dataset)
df = df.rename(columns={"image_path": "image_id"})
df["image_id"] = df["image_id"].str.replace("data/images/", "", regex=False)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

df.to_csv(os.path.join(data_root, "dataset.csv"), index=False)

label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [1]:
# To load the dataset again:
import pandas as pd
df = pd.read_csv('data/dataset.csv')

In [2]:
df["label"].value_counts()

0    522
2    518
3    514
4    505
1    462
Name: label, dtype: int64

In [3]:
import os

import pandas as pd
import torch
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from dataset import Dataset


In [4]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"])

# Change the path to the directory where the images are stored
path = "data/images"
train_dataset = Dataset(train_df, path)
test_dataset = Dataset(test_df, path)
val_dataset = Dataset(val_df, path)

In [6]:
batch_size = 16
lr = 2e-4
num_epochs = 35
num_classes = 5

from model import SoyaTrans
from train import Trainer


def run_experiment(batch_size, lr):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = SoyaTrans(num_classes)
    trainer = Trainer(model, train_loader, val_loader, test_loader, lr, num_epochs, batch_size=batch_size)

    trainer.train()
    trainer.test()
    torch.save(trainer.model.state_dict(), 'soyatrans.pth')

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/.local/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [7]:
# import itertools
# for batch_size, lr in itertools.product(batch_sizes, lrs):
    # print(f"\nRunning experiment with batch_size={batch_size}, lr={lr}")
    # run_experiment(batch_size, lr)

run_experiment(batch_size, lr)

/usr/lib/python3/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/lib/python3/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/lib/python3/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ./aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Epoch 1/35: 100%|██████████| 126/126 [00:54<00:00,  2.31it/s, loss=1.2807]



Train Metrics:
--------------------------------------------------
Epoch: 0
Train Loss: 1.5114
Test Loss: 1.4444
Accuracy: 0.4524
Precision: 0.2720
Recall: 0.4524
F1: 0.3386
--------------------------------------------------


Epoch 2/35: 100%|██████████| 126/126 [00:53<00:00,  2.36it/s, loss=1.2256]



Train Metrics:
--------------------------------------------------
Running experiment with batch_size=16, lr=0.0002
Epoch: 1
Train Loss: 1.3717
Test Loss: 1.3071
Accuracy: 0.6190
Precision: 0.6675
Recall: 0.6190
F1: 0.5868
--------------------------------------------------


Epoch 3/35: 100%|██████████| 126/126 [00:53<00:00,  2.36it/s, loss=1.2547]


...


Train Metrics:
--------------------------------------------------
Epoch: 32
Train Loss: 0.9212
Test Loss: 0.9533
Accuracy: 0.9484
Precision: 0.9489
Recall: 0.9484
F1: 0.9485
--------------------------------------------------


Epoch 34/35: 100%|██████████| 126/126 [00:55<00:00,  2.29it/s, loss=0.9066]



Train Metrics:
--------------------------------------------------
Epoch: 33
Train Loss: 0.9204
Test Loss: 0.9528
Accuracy: 0.9524
Precision: 0.9528
Recall: 0.9524
F1: 0.9525
--------------------------------------------------


Epoch 35/35: 100%|██████████| 126/126 [00:54<00:00,  2.30it/s, loss=0.9097]



Train Metrics:
--------------------------------------------------
Epoch: 34
Train Loss: 0.9198
Test Loss: 0.9539
Accuracy: 0.9524
Precision: 0.9527
Recall: 0.9524
F1: 0.9524
--------------------------------------------------

Test Metrics:
--------------------------------------------------
Test Loss: 0.9522
Accuracy: 0.9563
Precision: 0.9572
Recall: 0.9563
F1: 0.9564
--------------------------------------------------
